In [1]:
#Import Liabraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
ds=pd.read_csv("Cleaned_data")
ds

,Unnamed: 0,Location,Area_sqft,Road_Access,Electricity_Access,Water_Access,Distance_to_City_km,Nearby_Schools,Price_INR_Lakhs
0,0,Rural,2527.0,Street,0.0,0.0,32.12,8,105.30
1,1,Urban,3195.0,Street,1.0,1.0,3.22,3,82.48
2,2,Rural,1995.0,Highway,0.0,1.0,19.36,3,113.74
3,3,Rural,891.0,Street,0.0,1.0,31.67,5,137.41
4,4,Urban,4974.0,Street,0.0,0.0,25.65,2,89.85
...,...,...,...,...,...,...,...,...,...
85,95,Urban,3897.0,Main Road,0.0,0.0,49.35,2,115.19
86,96,Urban,3011.0,Street,1.0,0.0,19.34,0,82.03
87,97,Suburban,3146.9,Highway,0.0,1.0,19.16,4,109.47
88,98,Urban,3477.0,Highway,0.0,1.0,40.83,6,70.20


In [3]:
#Spliting data into indepedant and dependant 
X= ds[["Location",               
"Area_sqft",
"Road_Access",
"Electricity_Access",
"Water_Access",
"Distance_to_City_km",
"Nearby_Schools"]]
y=ds[['Price_INR_Lakhs']]

In [4]:
# implement OneHotEncoder

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
ohe=OneHotEncoder()
ohe.fit(X[["Location","Road_Access"]])

OneHotEncoder()

In [5]:
ct = make_column_transformer((OneHotEncoder(handle_unknown = 'ignore',categories=ohe.categories_),[ "Location","Road_Access"]),remainder='passthrough',force_int_remainder_cols=False,sparse_threshold = 0)
ct

ColumnTransformer(force_int_remainder_cols=False, remainder='passthrough',
                  sparse_threshold=0,
                  transformers=[('onehotencoder',
                                 OneHotEncoder(categories=[array(['Rural', 'Suburban', 'Urban'], dtype=object),
                                                           array(['Highway', 'Main Road', 'Street'], dtype=object)],
                                               handle_unknown='ignore'),
                                 ['Location', 'Road_Access'])])

In [6]:
# making pipeline

from sklearn.tree import DecisionTreeRegressor
regDTR = DecisionTreeRegressor(random_state = 0)

In [7]:
from sklearn.pipeline import make_pipeline
pipeDTR = make_pipeline(ct, regDTR)
pipeDTR

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(force_int_remainder_cols=False,
                                   remainder='passthrough', sparse_threshold=0,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Rural', 'Suburban', 'Urban'], dtype=object),
                                                                            array(['Highway', 'Main Road', 'Street'], dtype=object)],
                                                                handle_unknown='ignore'),
                                                  ['Location',
                                                   'Road_Access'])])),
                ('decisiontreeregressor',
                 DecisionTreeRegressor(random_state=0))])

In [8]:
# Splitting dataset into training and testing

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
scores = []
for i in range(0,101):
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.02,random_state= i)
    pipeDTR.fit(X_train,y_train.values.flatten())
    result=pipeDTR.predict(X_test)
    score = r2_score(y_test,result)
    scores.append(score)

In [9]:
# Finding best value of Decision Tree Reg

bestindex = np.argmax(scores)
scores[bestindex]

0.9882483693435283

In [10]:
# predict for user_input

Location=input("enter Location name : ")
Area_sqft=int(input("enter Area_sqft name : "))
Road_Access=input("enter Road_Access : ")
Electricity_Access=int(input("enter Electricity_Access : "))
Water_Access=int(input("enter Water_Access : "))
Distance_to_City_km=int(input("enter Distance_to_City_km:"))
Nearby_Schools=int(input("enter Nearby_Schools : "))
columns=["Location","Area_sqft","Road_Access","Electricity_Access","Water_Access","Distance_to_City_km","Nearby_Schools"]
myinput=pd.DataFrame(columns=columns,data=[[Location,Area_sqft,Road_Access,Electricity_Access,Water_Access,Distance_to_City_km,Nearby_Schools]])
result=pipeDTR.predict(myinput)
print("You should buy it for ~ price : ",abs(round(result[0])),"lakhs")

enter Location name :  Rural
enter Area_sqft name :  2234
enter Road_Access :  Street
enter Electricity_Access :  1
enter Water_Access :  1
enter Distance_to_City_km: 21
enter Nearby_Schools :  3


You should buy it for ~ price :  97 lakhs


In [11]:
import pickle
with open('model.pkl', 'wb') as f:
    pickle.dump(pipeDTR, f)